# **DECORATORS**

* ***A callable that -- takes a callable as input and -- returns another callable***
  
  
* Extends and modifies the input callable (functions, methods, classes)   
without permanently modifying the callable itself
  
  
* Use cases such as:
  * logging
  * enforcing access control and authentication
  * instrumentation and timing functions
  * rate-limiting
  * caching, and more
    
    
* Highly used in Python standard library and third-party frameworks

# **The syntax**

In [6]:
def null_decorator(func):
  return func

### METHOD 1:

- Explicitly calling the decorator on the function
- If you'll later need to call the undecorated function

In [ ]:
def greet():
  return 'Hello!'

greet = null_decorator(greet)
greet()

### METHOD 2:
- Decorating the function at definition time with the @ syntax
- Difficult to access the wrapped function later

In [50]:
@null_decorator
def greet():
  return 'Hello!'

greet()

'Hello!'

# **Example**

- Defining a decorator function

In [3]:
def uppercase(func):

  # closure used to wrap input function and modify its behavior
  def wrapper():
    original_result = func()
    modified_result = original_result.upper()
    return modified_result

  return wrapper

- Using the decorator

In [4]:
@uppercase
def greet():
  return 'Hello!'

greet()

'HELLO!'

In [7]:
# Decorator returns a different function object when it decorates a function
def null_decorator(func):
  return func

print(greet)  
print(null_decorator(greet))  
print(uppercase(greet))  

<function uppercase.<locals>.wrapper at 0x10eb87320>
<function uppercase.<locals>.wrapper at 0x10eb87320>
<function uppercase.<locals>.wrapper at 0x10f4333b0>


* Only way to influence the "future behavior" of the input function is to replace (***wrap***) it with a closure
  
  
* Decorator:
  * defines and returns another function (a closure)
  * that can be called at a later time,
  * to run the original input function
  * and modify its result
   
   
* In other words, decorators modify the behavior of a callable through a wrapper closure  
so you don't have to permanently modify the original (its behavior changes only when decorated)

# **Applying multiple decorators**

* __Decorator stacking__: applied from top to bottom
* Can have an effect on performance, only a problem for performance-intensive projects

In [54]:
def strong(func):
  def wrapper():
    return '<strong>' + func() + '</strong>'
  return wrapper

def emphasis(func):
  def wrapper():
    return '<em>' + func() + '</em>'
  return wrapper

# Same as: decorator_greet = strong(emphasis(greet))
@strong
@emphasis
def greet():
  return 'Hello!'

greet()

'<strong><em>Hello!</em></strong>'

# **Decorating functions that accept arguments**
* Closure uses `*args` and `**kwargs` to collect all positional and keyword arguments
* Then `wrapper` closure forwards collected arguments to the original input function  
using the `*` and `**` operators for argument unpacking   
   
(The meaning of the star and double-star operators is overloaded and it changes depending on the context)

In [55]:
def proxy(func):
  def wrapper(*args, **kwargs):
    return func(*args, **kwargs)
  return wrapper

def trace(func):
  def wrapper(*args, **kwargs):
    print(f'TRACE: calling {func.__name__}() '
          f'with {args}, {kwargs}')
    
    original_result = func(*args, **kwargs)
    
    print(f'TRACE: {func.__name__}() '
          f'returned {original_result!r}')
    
    return original_result

  return wrapper

@trace
def say(name, line):
  return f'{name}: {line}'

say('Jane', 'Hello world')

TRACE: calling say() with ('Jane', 'Hello world'), {}
TRACE: say() returned 'Jane: Hello world'


'Jane: Hello world'

# **How to write debuggable decorators**
* CON of decorators: 
  - it **hides some metadata** from attached to the original (undecorated) function  
(the original function name, its docstring, its parameter list...)
* SOLUTION: 
  - use `func-tools.wraps` from standard library in __all__ your decorators  
to **copy lost metadata** from the undecorated function to the decorator closure

In [8]:
def uppercase(func):

  # closure used to wrap input function and modify its behavior
  def wrapper():
    original_result = func()
    modified_result = original_result.upper()
    return modified_result

  return wrapper

def greet():
  '''Return a friendly greeting.'''
  return 'Hello!'

decorated_greet = uppercase(greet)

print('* Undecorated function:')
print('         name: ', greet.__name__)  
print('    docstring: ', greet.__doc__)  
print()
print('* Decorated function:')
print('         name: ', decorated_greet.__name__)  
print('    docstring: ', decorated_greet.__doc__)  

* Undecorated function:
         name:  greet
    docstring:  Return a friendly greeting.

* Decorated function:
         name:  wrapper
    docstring:  None


In [9]:
import functools

def uppercase(func):
  @functools.wraps(func)
  def wrapper():
    return func().upper()
  return wrapper

@uppercase
def greet():
  '''Return a friendly greeting.'''
  return 'Hello!'

print(greet.__name__) 
print(greet.__doc__) 

greet
Return a friendly greeting.
